## Convert behavior data

In [1]:
from numpy import concatenate, cumsum, convolve, ones
from pandas import read_csv, concat
from glob import glob
from os.path import exists

###Set directory and session information

In [2]:
directory = '/Volumes/freeman/Nick/mVR/sessions'

Check if behavior data exists

In [3]:
key = '000131'
path = directory + '/' + key
print path
print exists(path)

/Volumes/freeman/Nick/mVR/sessions/000131
True


###Load trial data

In [4]:
files = glob(path + '/behavior/trial-*.csv')
files.sort()
trials = [read_csv(x) for x in files]

Create session table

In [5]:
session = concat(trials)
session.reset_index(inplace=True)
session.rename(columns={'index': 'tick'}, inplace=True)

Add global time

In [6]:
trial = session.groupby(['number']).last()
offsets = concatenate(([0],trial.time.values))
offsets = cumsum(offsets)
session['allTime'] = session.time + offsets[session.number]

Add filtered speed

In [7]:
speed = (session.xSpeed.pow(2)+session.ySpeed.pow(2)).pow(.5)
session['speed'] = convolve(speed,ones(50)/50, mode='same')    

### Save full-length session

In [8]:
session.to_csv(path + '/behavior/session.csv')